# National Weather Service 24hr Accumulated Rainfall
## Justin Richling 4/17/2017

## http://water.weather.gov/precip/download.php

In [ ]:
%pylab
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import time,urllib,urllib2,cStringIO,logging,datetime,webbrowser,\
IPython.display,shutil,sys, os
import numpy as np
from mpl_toolkits.basemap import Basemap, cm
from scipy.io import netcdf
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from IPython.core.display import Image
from PIL import Image as PILImage

In [ ]:
import tarfile
 
def untar(tarFile):
    if (tarFile.endswith("tar.gz")):
        tar = tarfile.open(tarFile)
        tar.extractall()
        tar.close()
    else:
        print "Not a tar.gz file: '%s '" % sys.argv[0]


In [ ]:
ef Rain24hrTotal(year,month,day,Datepath,Downloadpath,region):
    if month < 10:
        month = "0"+str(month)
    if day < 10:
        day = "0"+str(day)
        
    # The NetCDF file come named : nws_precip_+str(Year)+"_"+str(Month)+"_"+str(Day)+_nc.tar
    untar(Downloadpath+"nws_precip_"+Datepath+"_nc.tar.gz")
    # The untaring will produce (usually) .nc files for: 
    # Cont. US (conus)
    # Alaska (ak)
    # Puerto Rico (pr)
    
    # plot rainfall from NWS using special precipitation
# colormap used by the NWS, and included in basemap.
  
    # netcdf.netcdf_file() will save the archived .nc file in desired location
    nc = netcdf.netcdf_file(Downloadpath+"nws_precip_"+region+"_"+Datepath+".nc")

    PrecipFilename = str(Year)+"_"+str(Month)+"_"+str(Day)+"_nws_precip.png"
# data from http://water.weather.gov/precip/
    prcpvar = nc.variables['amountofprecip']
    data = 0.01*prcpvar[:]
    latcorners = nc.variables['lat'][:]
    loncorners = -nc.variables['lon'][:]
    lon_0 = -nc.variables['true_lon'].getValue()
    lat_0 = nc.variables['true_lat'].getValue()
    nc.close()
# create figure and axes instances
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
# create polar stereographic Basemap instance.
    m = Basemap(projection='stere',lon_0=lon_0,lat_0=90.,lat_ts=lat_0,\
            llcrnrlat=latcorners[0],urcrnrlat=latcorners[2],\
            llcrnrlon=loncorners[0],urcrnrlon=loncorners[2],\
            rsphere=6371200.,resolution='l',area_thresh=10000)
# draw coastlines, state and country boundaries, edge of map.
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
# draw parallels.
    parallels = np.arange(0.,90,10.)
    m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
    meridians = np.arange(180.,360.,10.)
    m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
    ny = data.shape[0]; nx = data.shape[1]
    # get lat/lons of ny by nx evenly space grid.
    lons, lats = m.makegrid(nx, ny) 
    x, y = m(lons, lats) # compute map proj coordinates.
# draw filled contours.
    clevs = [0,1,2.5,5,7.5,10,15,20,30,40,50,70,100,150,\
             200,250,300,400,500,600,750]
    cs = m.contourf(x,y,data,clevs,cmap=cm.s3pcpn)
# add colorbar.
    cbar = m.colorbar(cs,location='bottom',pad="5%")
    cbar.set_label('mm')
# add title
    plt.title(prcpvar.long_name+' for period ending '+\
              prcpvar.dateofdata)
    plt.savefig(Downloadpath+PrecipFilename)
    #plt.axis('off')
    #plt.show()
    plt.close()

In [ ]:
now = datetime.datetime.now()
Time = raw_input("(C)urrent time or (a)rchive? ")
if Time == "C": 
    Day = now.day
    Year = now.year
    Month = now.month
    
if Time == "a":
    Year = input("Year: ")
    Month = input("Month (numerical 1-12): ")
    Day = input("Day (numerical 1-28/29/30/31): ")

Downloadpath = raw_input("Where would you like the file? \
(For now you need to provide the full path file)")    
Datepath = str(Year)+str(Month)+str(Day)

if not os.path.isdir(Downloadpath):
    os.makedirs(Downloadpath)
os.chdir(Downloadpath)

webbrowser.open("http://water.weather.gov/precip/download.php")
LetsRoll = raw_input("Downloaded? (y) ")
if LetsRoll == "y":
    region = raw_input("Which region would you like? (conus, ak, pr) ")
    Rain24hrTotal(Year,Month,Day,Datepath,Downloadpath,region)
    
# To automatically put the .tar file in the trash (on Mac OSX)
    #shutil.move(Downloadpath+"nws_precip_"+\
    #str(Year)+str(Month)+str(Day)+"_nc.tar.gz"\
    #, "/Users/UserName/.Trash/nws_precip_"+\
    #str(Year)+str(Month)+str(Day)+"_nc.tar.gz")